In [1]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
       # if item.abbrev == 'A' or item.abbrev == 'I':
          #  item.abbrev = '-'
        tickers.append(item)

In [2]:
import pandas as pd

submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [3]:
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [4]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [5]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif..."
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,..."
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,..."
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]"
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]"


In [6]:
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            #if (ticker.name.split()[0] in body_set) or (ticker.name.split()[0] in title_set):
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                #print("Abbreviation match")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                #print("Name match")
                #print(f"{index}: {ticker.name}")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    submissions_df.at[index, "Tickers"]= tickers_found
    print(f"{index}: {tickers_found}")

0: ['MDP']
1: []
2: ['AAPL', 'AES', 'AM', 'AMZN', 'BLL', 'CEL', 'CRWD', 'DG', 'DPZ', 'ET', 'F', 'FB', 'FDX', 'FOR', 'G', 'GM', 'GPN', 'HQY', 'IVZ', 'LPL', 'MCK', 'MKTX', 'MTD', 'NDAQ', 'NEXT', 'NKE', 'OPEN', 'PM', 'ROST', 'TREE', 'TSLA', 'TWOA', 'U', 'V', 'VFF', 'VUZI', 'W', 'QQQ']
3: ['AMD', 'DD', 'INTC', 'NVDA', 'PE', 'QCOM', 'TSM', 'TWOA', 'UMC']
4: ['CODX']
5: ['TSLA']
6: ['HMNY', 'WISH', 'ARKK']
7: ['TSLA']
8: ['DD', 'RKT']
9: ['AONE', 'B']
10: ['ATH']
11: []
12: []
13: []
14: ['CRM', 'E', 'TA', 'QQQ']
15: []
16: ['CCL', 'CUK']
17: []
18: ['GME', 'HOME', 'OR', 'WWW']
19: ['AAPL', 'AMZN', 'APHA', 'CHPT', 'DIS', 'ARKF', 'PRNT']
20: []
21: ['B', 'DD', 'SC', 'TRIT']
22: ['ABBV', 'ARAV', 'ARDX', 'ATXI', 'AVB', 'AVEO', 'BBIO', 'BIOT', 'BLUE', 'BMY', 'COR', 'FBIO', 'KN', 'KNSA', 'LPTX', 'MRK', 'NWS', 'NWSA', 'PCRX', 'PFE', 'PLX', 'RTX', 'RUBY', 'RYTM', 'SBBP', 'SLDB', 'SLGL', 'SPN', 'SPRB', 'SRPT', 'SUPN', 'UTHR', 'ZEAL']
23: ['CEO', 'DD', 'LMND', 'CFO']
24: ['TWOA', 'ITM']
25: ['AONE', 

In [7]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP]
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[]
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,..."
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]"
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX]


In [8]:
import nltk
#nltk.download('punkt')

In [9]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_df['negative_score'] = ""
submissions_df['neutral_score'] = ""
submissions_df['positive_score'] = ""
submissions_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in submissions_df.iterrows():
    title = row[2]
    body = row[4]
    eval_text = title + " " + body
    body_words = row[12]
    title_words = row[13]
    tickers = row[14]
    pol_score= sia.polarity_scores(eval_text)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    submissions_df.at[index, 'negative_score'] = pol_score['neg']
    submissions_df.at[index, 'neutral_score'] = pol_score['neu']
    submissions_df.at[index, 'positive_score'] = pol_score['pos']
    submissions_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],0,0.898,0.102,0.5632
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[],0.095,0.739,0.166,0.9882
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...",0.032,0.879,0.089,0.9998
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]",0.042,0.894,0.064,0.9386
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],0.011,0.878,0.111,0.9626


In [10]:
submissions_df['sentiment_label'] = 0
submissions_df.loc[submissions_df['compound_score'] > 0.2, 'sentiment_label'] = 1
submissions_df.loc[submissions_df['compound_score'] < -0.2, 'sentiment_label'] = -1
submissions_df

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],0,0.898,0.102,0.5632,1
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[],0.095,0.739,0.166,0.9882,1
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...",0.032,0.879,0.089,0.9998,1
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]",0.042,0.894,0.064,0.9386,1
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],0.011,0.878,0.111,0.9626,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Submission,m506f3,Tesla FSD: Am I the only one who thinks this way?,GordonGekkoVienna,"First of all, I do not own put options on Tesl...",NaN,16,t3_m506f3,https://www.reddit.com/r/stocks/comments/m506f...,0,0.50,[1615744686.0],"[First, of, all, I, do, not, own, put, options...","[Tesla, FSD, Am, I, the, only, one, who, think...",[TSLA],0.05,0.882,0.068,0.5306,1
90,Submission,m4y62w,How to play through a potential long term hold...,Alternative-Swan9228,Hey dudes. Just wondering if anyone got into $...,NaN,7,t3_m4y62w,https://www.reddit.com/r/stocks/comments/m4y62...,0,0.44,[1615738829.0],"[Hey, dudes, Just, wondering, if, anyone, got,...","[How, to, play, through, a, potential, long, t...",[],0.048,0.792,0.16,0.9683,1
91,Submission,m4st3i,Looking for an honest opinion,xxthey,"I’m new to this, but I think of stocks on a lo...",NaN,16,t3_m4st3i,https://www.reddit.com/r/stocks/comments/m4st3...,0,0.25,[1615717764.0],"[I, m, new, to, this, but, I, think, of, stock...","[Looking, for, an, honest, opinion]","[B, TWOA]",0,0.802,0.198,0.9754,1
92,Submission,m51dh3,Is anyone just tired and want to wait for the ...,thingsfakerdoes,I started investing during last Feb when the p...,NaN,26,t3_m51dh3,https://www.reddit.com/r/stocks/comments/m51dh...,0,0.33,[1615748032.0],"[I, started, investing, during, last, Feb, whe...","[Is, anyone, just, tired, and, want, to, wait,...",[SPY],0.065,0.795,0.14,0.9627,1


In [11]:
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0]
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0]
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0]
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0]
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0]


In [12]:
comments_df['Body_Words'] = comments_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')

In [13]:
comments_df['Tickers'] = ""
for index_c, row_c in comments_df.iterrows():
    for index_s, row_s in submissions_df.iterrows():
        if row_c[1] == row_s[1]:
             comments_df.at[index_c, 'Tickers'] = row_s[14]

In [14]:
comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP]
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP]
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP]
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP]
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP]


In [19]:
comments_df['negative_score'] = ""
comments_df['neutral_score'] = ""
comments_df['positive_score'] = ""
comments_df['compound_score'] = ""
#submissions_loop = submissions_df[['Title', 'Body', 'Body_Words', 'Title_Words', 'Tickers']]
for index, row in comments_df.iterrows():
    body = row[5]
    tickers = row[11]
    pol_score= sia.polarity_scores(body)
    #pol_score['submission_text'] = eval_text
    #pol_score['tickers'] = tickers
    #results.append(pol_score)
    comments_df.at[index, 'negative_score'] = pol_score['neg']
    comments_df.at[index, 'neutral_score'] = pol_score['neu']
    comments_df.at[index, 'positive_score'] = pol_score['pos']
    comments_df.at[index, 'compound_score'] = pol_score['compound']
        #submissions_loop['sentiment_scores'] = pol_score
#pprint(results[:3], width=100)

comments_df.head()

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP],0,0.953,0.047,0.296,0
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP],0.15,0.711,0.139,-0.1027,0
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP],0.031,0.908,0.062,0.25,0
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP],0,0.8,0.2,0.3612,0
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP],0,1,0,0,0


In [21]:
comments_df['sentiment_label'] = 0
comments_df.loc[comments_df['compound_score'] > 0.2, 'sentiment_label'] = 1
comments_df.loc[comments_df['compound_score'] < -0.2, 'sentiment_label'] = -1
comments_df

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
0,Comment,m4inen,gquodoo,t3_m4inen,wangkerd,"[""Because the stock has split twice since 1988...",NaN,False,2327,[1615681875.0],"[, Because, the, stock, has, split, twice, sin...",[MDP],0,0.953,0.047,0.296,1
1,Comment,m4inen,gqujupq,t3_m4inen,ExtremeNihilism,"[""Holy shit, I can't believe I'm in early to w...",NaN,False,4013,[1615679240.0],"[, Holy, shit, I, cant, believe, Im, in, early...",[MDP],0.15,0.711,0.139,-0.1027,0
2,Comment,m4inen,gqulrra,t3_m4inen,Aelsar,"[""Hey OP.\n\nYou've got about a million bucks ...",NaN,False,3402,[1615680362.0],"[, Hey, OP, n, nYouve, got, about, a, million,...",[MDP],0.031,0.908,0.062,0.25,1
3,Comment,m4inen,gquk24t,t3_m4inen,ItsTwentyBaby,['Looks like there was two stock splits after ...,NaN,False,472,[1615679359.0],"[, Looks, like, there, was, two, stock, splits...",[MDP],0,0.8,0.2,0.3612,1
4,Comment,m4inen,gqup8nu,t3_m4inen,FancyGonzo,['Lol gramps probably spent the last 10 years ...,NaN,False,117,[1615682360.0],"[, Lol, gramps, probably, spent, the, last, 10...",[MDP],0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,Comment,m53jxd,gqyj887,t1_gqygx9s,MatterLover1729,['Though not relevant to my comment on republi...,NaN,False,0,[1615765107.0],"[, Though, not, relevant, to, my, comment, on,...",[UK],0.094,0.816,0.09,-0.1687,0
3259,Comment,m53jxd,gqy1yy3,t1_gqy1cjs,TheXorzox,['two of the biggest states and the cdc adding...,NaN,True,1,[1615756854.0],"[, two, of, the, biggest, states, and, the, cd...",[UK],0,1,0,0,0
3260,Comment,m53jxd,gqyjwqg,t1_gqyj887,TheXorzox,['i understand just fine. When democrats cage ...,NaN,True,2,[1615765486.0],"[, i, understand, just, fine, When, democrats,...",[UK],0,0.847,0.153,0.2023,1
3261,Comment,m53jxd,gqykfp3,t1_gqyjwqg,MatterLover1729,['I understand you cannot read'],NaN,False,1,[1615765781.0],"[, I, understand, you, cannot, read, ]",[UK],0,1,0,0,0


In [22]:
comments_df.loc[comments_df['Submission_ID'] == 'm50u8p']

,Post_Type,Submission_ID,Comment_ID,Parent_ID,Author,Body,Distinguished,Is_Author,Score,Created_Date_UTC,Body_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
1151,Comment,m50u8p,gqxakdy,t3_m50u8p,Boomtown626,"[""They're cheap because you need the thing to ...",NaN,False,27,[1615746845.0],"[, Theyre, cheap, because, you, need, the, thi...",[CODX],0.046,0.932,0.023,-0.2732,-1
1152,Comment,m50u8p,gqxkl45,t3_m50u8p,1ThoughtMaze1,['It’s all about the vaccines now so I think t...,NaN,False,11,[1615750446.0],"[, It, s, all, about, the, vaccines, now, so, ...",[CODX],0,1,0,0,0
1153,Comment,m50u8p,gqxkges,t3_m50u8p,mightylfc,"[""You're way too leveraged on this! Hope it pa...",NaN,False,11,[1615750400.0],"[, Youre, way, too, leveraged, on, this, Hope,...",[CODX],0,0.728,0.272,0.8776,1
1154,Comment,m50u8p,gqxzxmc,t3_m50u8p,MoarTarriffs,['Lol stop trying to pump codx with fake up up...,NaN,False,7,[1615756143.0],"[, Lol, stop, trying, to, pump, codx, with, fa...",[CODX],0.331,0.558,0.112,-0.7096,-1
1155,Comment,m50u8p,gqxob5z,t3_m50u8p,rbmrph,"[""There isn't much liquidity in those options....",NaN,False,3,[1615751779.0],"[, There, isnt, much, liquidity, in, those, op...",[CODX],0,0.615,0.385,0.7579,1
1156,Comment,m50u8p,gqxxs10,t3_m50u8p,xhavez,['What happened in August?'],NaN,False,3,[1615755330.0],"[, What, happened, in, August, ]",[CODX],0,1,0,0,0
1157,Comment,m50u8p,gqxpf2f,t3_m50u8p,SirGamer247,['definitely adding to my watch list'],NaN,False,2,[1615752165.0],"[, definitely, adding, to, my, watch, list, ]",[CODX],0,1,0,0,0
1158,Comment,m50u8p,gqxeu9t,t3_m50u8p,Scared_Patience_4802,['Also just found this news article from yeste...,NaN,True,2,[1615748358.0],"[, Also, just, found, this, news, article, fro...",[CODX],0,1,0,0,0
1159,Comment,m50u8p,gqxkxcl,t3_m50u8p,Equivalent_Gazelle41,"[""CODX is legit. I was into it last year as a ...",NaN,False,2,[1615750564.0],"[, CODX, is, legit, I, was, into, it, last, ye...",[CODX],0,0.771,0.229,0.9042,1
1160,Comment,m50u8p,gqyfhur,t3_m50u8p,Edjes,"[""What's the moat like?""]",NaN,False,1,[1615763037.0],"[, Whats, the, moat, like, ]",[CODX],0,1,0,0,0


In [36]:
compound_average = comments_df[['Submission_ID','sentiment_label']].groupby('Submission_ID').mean()

In [37]:
compound_average

,sentiment_label
Submission_ID,
m4i6p3,-0.058824
m4iaqz,0.500000
m4iiqk,0.272727
m4inen,0.280561
m4j2cb,0.370370
...,...
m55hqq,0.000000
m55ioo,0.500000
m55xx4,-0.166667


In [44]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,negative_score,neutral_score,positive_score,compound_score,sentiment_label
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],0,0.898,0.102,0.5632,1
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[],0.095,0.739,0.166,0.9882,1
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...",0.032,0.879,0.089,0.9998,1
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]",0.042,0.894,0.064,0.9386,1
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],0.011,0.878,0.111,0.9626,1
